In [544]:
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
import logging
import os
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

import nngp
import neural_net

from IPython.core.display import HTML

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Experimental Functions

In [25]:
def prep_data(X, Y, dtype=tf.float64):
    X_flat = tf.convert_to_tensor(X.reshape(-1, 28*28)/255, dtype=dtype)
    Y_cat = keras.utils.to_categorical(Y)
    Y_cat = Y_cat - 0.1
    Y_cat = tf.convert_to_tensor(Y_cat, dtype=dtype)
    return X_flat, Y_cat

In [26]:
def evaluate_NN(X_train, Y_train, X_test, Y_test, model):
    model.fit(X_train, Y_train, verbose=0)
    output = model.predict(X_test, verbose=0)
    predictions = tf.argmax(output, axis=1).numpy()
    acc = np.equal(predictions, Y_test).sum()/len(Y_test)
    return acc, loss

In [119]:
def evaluate_GP(X_train, Y_train, X_test, Y_test, K, jitter=0.1):
    mu_bar, _ = nngp.GP(X_train, Y_train, X_test, K, jitter)
    predictions = tf.argmax(mu_bar, axis=1).numpy()
    
    acc = np.equal(predictions, Y_test).sum()/len(Y_test)
    return acc

## Experiments

### NN vs NNGP

In [33]:
# Load the mnist dataset
(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()

In [34]:
X_train_flat, Y_train_cat = prep_data(X_train, Y_train)
X_test_flat, Y_test_cat = prep_data(X_test, Y_test)
din = X_train_flat.shape[1]
dout = Y_test_cat.shape[1]

In [62]:
n_data = 1000
n_test = 200

D = X_train_flat[:n_data]
t = Y_train_cat[:n_data]
X_test_n = X_test_flat[:n_test]
Y_test_n = Y_test[:n_test]

In [116]:
act = tf.nn.relu
general_kernel = nngp.GeneralKernel(act, L=4, n_g=401, n_v=400, n_c=400, u_max=10, s_max=100)

2020-04-24 23:32:31,626 - Caclulating grid
2020-04-24 23:32:31,638 - Loaded grid from file


In [106]:
analytical_kernel = nngp.AnalyticalKernel(L=1)
# analytical_kernel.K(D,X)

In [107]:
K_D_general = general_kernel.K(D, X_test_n)

In [108]:
K_D_analytical = analytical_kernel.K(D, X_test_n)

In [109]:
K_D_general

<tf.Tensor: shape=(1000, 200), dtype=float64, numpy=
array([[1.45721028, 1.45726183, 1.45614859, ..., 1.45946671, 1.45659993,
        1.46062993],
       [1.45594612, 1.45656417, 1.45504623, ..., 1.45810963, 1.45535751,
        1.45902862],
       [1.45836028, 1.45797673, 1.4573527 , ..., 1.45882156, 1.45849677,
        1.45944988],
       ...,
       [1.45591871, 1.45595254, 1.45468848, ..., 1.45820913, 1.45518637,
        1.45824654],
       [1.45755822, 1.45978671, 1.45739851, ..., 1.4582156 , 1.45724842,
        1.46057181],
       [1.45704513, 1.45861511, 1.45704676, ..., 1.46124066, 1.45726532,
        1.46002263]])>

In [110]:
(K_D_general-K_D_analytical).numpy().max()

-0.04195632307720176

In [126]:
acc = evaluate_GP(D, t, X_test_n, Y_test_n, general_kernel.K, jitter=0.01)

In [127]:
acc

0.885

In [128]:
acc = evaluate_GP(D, t, X_test_n, Y_test_n, analytical_kernel.K, jitter=0.01)

2020-04-24 23:33:16,033 - 7 out of the last 11 calls to <function _call_analytical_K at 0x1056e1170> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.


2020-04-24 23:33:16,212 - 7 out of the last 11 calls to <function _call_analytical_K at 0x1056e1170> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.


2020-04-24 23:33:16,357 - 7 out of the last 11 calls to <function _call_analytical_K at 0x1056e1170> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.


In [129]:
acc

0.955

In [107]:
model = build_model((din,), dout, 3, 2000, 'relu', final_activation=None)

In [108]:
model.compile(optimizer='Adam', loss='MSE', metrics=['accuracy'])

In [ ]:
evaluate_NN(D, t, X, Y_test, model)

In [159]:
fig = go.Figure(
    data=[go.Scatter(y=mu_bar, x=x)],
    layout_title_text="A Figure Displayed with fig.show()"
)
fig.write_html("file.html")

NameError: name 'mu_bar' is not defined

In [161]:
import tensorflow_probability as tfp

In [ ]:
yref = [[10,20,30],[20,30,40]]
yref = tf.convert_to_tensor(yref)
x_ref_min = tf.constant([0, 0])
x_ref_max = tf.constant([3, 3])
x = 


tfp.math.batch_interp_regular_nd_grid(
    x, x_ref_min, x_ref_max, y_ref, axis, fill_value='constant_extension', name=None
)

In [207]:
x_ref_min = [0., 0.]
x_ref_max = [2 * np.pi, 2 * np.pi]

# Build y_ref.
x0s, x1s = tf.meshgrid(
    tf.linspace(x_ref_min[0], x_ref_max[0], num=100),
    tf.linspace(x_ref_min[1], x_ref_max[1], num=100),
    indexing='ij')

def func(x0, x1):
    return tf.sin(x0) * tf.cos(x1)

y_ref = func(x0s, x1s)

x = np.pi * tf.random.uniform(shape=(10, 2))

tfp.math.batch_interp_regular_nd_grid(x, x_ref_min, x_ref_max, y_ref, axis=-2)

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([-0.1416741 ,  0.5625572 , -0.48798943,  0.7856882 ,  0.08622341,
        0.304323  ,  0.72483563,  0.16174427,  0.17111623,  0.46674064],
      dtype=float32)>

In [208]:
x

<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[2.8638003 , 2.114081  ],
       [2.1672916 , 0.8227545 ],
       [0.95602506, 2.2112918 ],
       [1.6944642 , 0.65602136],
       [0.09083032, 0.31378147],
       [2.3476958 , 1.1297177 ],
       [2.1021342 , 0.57177037],
       [2.6720374 , 1.2051793 ],
       [1.3167788 , 1.3929853 ],
       [0.57677644, 0.54241467]], dtype=float32)>

In [365]:
x_ref_min = 0
x_ref_max = 10
Xp = tf.convert_to_tensor([3.4], dtype=tf.float64)
X = tf.convert_to_tensor(np.linspace(10,20,10), dtype=tf.float64)
elements = tfp.math.batch_interp_regular_1d_grid(Xp, 0, 10, X)

In [366]:
elements

<tf.Tensor: shape=(1,), dtype=float64, numpy=array([13.4])>

### Angular form of the kernel

In [481]:
def make_norms(vectors):
    """ Makes square of the norm of a vector equal it's dimension
    """
    din = len(vectors[0])
    return np.sqrt(din)*vectors / np.linalg.norm(vectors, axis=1, keepdims=True)

def rotate(vector, theta):
    """ Rotate a 2d vector by an angle theta
    """
    R = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
    return np.matmul(R, vector.T)

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

In [496]:
n_vectors = 50

In [497]:
# Generate n_vector pairs at a range of angles
angles = np.linspace(0, 2*np.pi, n_vectors)
vectors = np.random.random(size=(n_vectors, 2))
rotated_vectors = np.empty(vectors.shape)
for i, vector in enumerate(vectors):
    rotated_vectors[i,:] = rotate(vector, angles[i])
    
vectors = make_norms(vectors)
rotated_vectors = make_norms(rotated_vectors)

In [499]:
def run_angular(vectors, rotated_vectors, K):
    """ Obtain result of applying kernel K to vectors and rotated_vetors in turn
    """

    K_thetas = np.empty((vectors.shape[0],))
    for i, vector_pair in enumerate(zip(vectors, rotated_vectors)):
        K_thetas[i] = K(tf.convert_to_tensor([vector_pair[0]], dtype=tf.float64), 
                        tf.convert_to_tensor([vector_pair[1]], dtype=tf.float64)).numpy()

    return np.squeeze(K_thetas)

In [500]:
depths = range(0, 9)
sigma_b = np.sqrt(0.1)
sigma_w = np.sqrt(1.6)

# Intialise Kernels
analytical_kernel = nngp.AnalyticalKernel(L=0, sigma_b=sigma_b, sigma_w=sigma_w)
general_kernel = nngp.GeneralKernel(tf.nn.relu, sigma_b=sigma_b, sigma_w=sigma_w ,L=0, n_g=401, n_v=400, n_c=400, u_max=10, s_max=100)

K_thetas_analytical = np.empty((len(vectors), len(depths)))
K_thetas_general = np.empty((len(vectors), len(depths)))

for depth in depths:
    analytical_kernel.L = depth
    general_kernel.L = depth
    K_thetas_analytical[:, depth] = run_angular(vectors, rotated_vectors, analytical_kernel.K)
    K_thetas_general[:, depth] = run_angular(vectors, rotated_vectors, general_kernel.K)

2020-04-25 12:11:42,526 - Caclulating grid
2020-04-25 12:11:42,536 - Loaded grid from file


In [531]:
import bokeh.io
from bokeh.plotting import figure, output_notebook, show
# this is here only for completeness to clarify where
# the methods are nested (you probably already imported this earlier)


bokeh.io.reset_output()
bokeh.io.output_notebook()

Loading BokehJS ...

In [543]:

# prepare some data
x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

# create a new plot with a title and axis labels
p = figure(title="Kernel Comparison", x_axis_label='x', y_axis_label='y', plot_width=800, plot_height=500)
# add a line renderer with legend and line thickness
for depth in depths:
    p.line(angles, K_thetas_analytical[:,depth], legend_label="Temp.", line_width=2, line_color='blue')
    p.scatter(angles, K_thetas_general[:,depth], legend_label="Interpolation", fill_color='red')

# show the results
show(p)

In [545]:
HTML('lines.html')